In [13]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
import cmath

from processing_func import from_S_to_T, from_T_to_S
from file_opener import run

In [14]:
%matplotlib widget
import matplotlib.pyplot as plt

Импорт данных

In [15]:
reflecting_coef_30deg_TE = pd.read_csv('ND_struct_30deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

reflecting_coef_0deg_TE = pd.read_csv('ND_struct_0deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

reflecting_coef_45deg_TE = pd.read_csv('ND_struct_45deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

reflecting_coef_60deg_TE = pd.read_csv('ND_struct_60deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

In [20]:
mes_list = [reflecting_coef_0deg_TE, reflecting_coef_30deg_TE, reflecting_coef_45deg_TE, reflecting_coef_60deg_TE]

data_25GHz = mes_list[0].loc[mes_list[0]['Frequency'] == 25]

for i in range(1, len(mes_list), 1):
    data_25GHz = pd.concat([data_25GHz, mes_list[i].loc[mes_list[i]['Frequency'] == 25]], ignore_index=True)

In [21]:
data_25GHz.head()

,Frequency,ReSz11,ImSz11
0,25.0,0.904027,0.050718
1,25.0,0.808577,-0.338870
2,25.0,0.833809,0.182458
3,25.0,0.748441,-0.184718
